In this worksheet, the following are calculated:
1. Plot on map location of viable brackish water sites
2. Plot on map location of power plants 
3. % demand of a powerplant that can be met by all brackish wells within a state
4. % demand of a powerplant that can be met without exceeding a maximum LCOW


In [ ]:
max_LCOW = 1.6 #$/m3
transport_cost = 0.0115   #$/m3/km

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import haversine as hs
import geopandas as gp
from shapely.geometry import Point, Polygon
import numpy as np

from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from pyomo.environ import Block, Expression, units as pyunits
import math
from truck_pipe_cost_functions import truck_costing, pipe_costing

Map brackish water sites that meet the following criteria
1. Well yield > 0.01
2. 0.5< TDS< 25 in kg/m3
3. Contiguous US states

LCOW calculated currently without transportation costs
File - brackish_sites_with_metrics

In [ ]:
# bw_df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_with_metrics.csv')
bw_df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_with_metrics_baseline_dwi_updated_costs_transport_updated_basis.csv')
netl_df = pd.read_csv("/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Power/NETL_ThermalPlants_Filtered.csv")

Updating powerplant dataframe to group demand with the same plant code (independent of fuel)\
Filter powerplants where freshwater is used and where all the data is available --> this filter is copied from previous files

In [ ]:
pp_df = netl_df[((netl_df.WaterType == 'Fresh') & (netl_df.WAvg != '...') & (netl_df.WAvg != '---'))]
pp_group= pp_df.groupby('PlantCode')
pp_df.WAvg = pp_df.WAvg.astype(float)
cols= ['PlantCode','Demand_m3/s','PPLatitude','PPLongitude']

pp_combined_df = pd.DataFrame(columns = cols)
pp_group= pp_df.groupby('PlantCode')

for key in pp_group.groups.keys():

    demand = sum((pp_group.get_group(key).WAvg * 0.0037854 / 3600) * pp_group.get_group(key).Capacity)
    # pp_combined_df = pp_combined_df.append
    temp_dict = {'PlantCode':key,
                 'PlantState':pp_group.get_group(key)['PlantState'].values[0],
                 'Demand_m3/s': demand ,
                 'PPLatitude':pp_group.get_group(key)['Latitude'].values[0],
                 'PPLongitude':pp_group.get_group(key)['Longitude'].values[0]}
    
    temp = pd.DataFrame(temp_dict,index= [0])                  
    pp_combined_df = pd.concat([pp_combined_df,temp],ignore_index=True)
    pp_combined_df.reset_index()


A sample map for Texas

In [ ]:
pp_combined_df

In [ ]:
texas = gp.read_file('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/GeoData/Texas Counties Map/geo_export_c05b3355-1638-4e46-b56c-ac18de751ed8.shp')
texas = texas.to_crs("EPSG:4326")


In [ ]:
bw_long = bw_df[bw_df['state_alpha']=='TX']['Longitude']
bw_lat = bw_df[bw_df['state_alpha']=='TX']['Latitude']

bw_geometry = [Point(xy) for xy in zip(bw_long,bw_lat)]
bw_geo_df = gp.GeoDataFrame(geometry = bw_geometry, crs='EPSG:4326')
bw_geo_df.geometry = bw_geo_df.geometry.to_crs('EPSG:4326')
bw_geo_df = bw_geo_df.assign(county_nm = bw_df[bw_df['state_alpha'] =='TX']['county_nm'].values)

pp_long = pp_combined_df[pp_combined_df['PlantState']=='TX']['PPLongitude']
pp_lat = pp_combined_df[pp_combined_df['PlantState']=='TX']['PPLatitude']

pp_geometry = [Point(xy) for xy in zip(pp_long,pp_lat)]
pp_geo_df = gp.GeoDataFrame(geometry = pp_geometry, crs='EPSG:4326')
pp_geo_df.geometry = pp_geo_df.geometry.to_crs('EPSG:4326')


In [ ]:
fig, ax = plt.subplots()
# states[states['NAME'] == 'Texas'].plot(ax=ax,figsize=(20, 20))
texas.plot(ax=ax,figsize=(50, 50))
bw_geo_df.plot(ax=ax, color='purple', markersize = 5,label = "BW Sites")
pp_geo_df.plot(ax=ax, color='black', markersize = 5,label='PP')

ax.legend()

Selecting brackish water sites for powerplants
1. Minimize cost for the state
2. Does not exceed the capacity of each brackish water site

In [ ]:
# Create matrix of brackish water sites (rows) and powerplant (columns) distance
from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from pyomo.util.infeasible import *
from idaes.core.util.model_statistics import *

bw_state_df = bw_df[bw_df['state_alpha']=='TX']
bw_state_df = bw_state_df.set_index('unique_site_ID')

pp_state_df =  pp_combined_df[pp_combined_df['PlantState']=='TX']
pp_state_df = pp_state_df.set_index('PlantCode')

no_bw_sites = len(bw_state_df)
no_pp_sites = len(pp_state_df)

dist = np.zeros((no_bw_sites,no_pp_sites))
pp_demand = []
# Create array with brackish water availability
bw_vol = []
LCOW = []
plantcode=[]

i = 0
j = 0 

for pp in pp_state_df.index:
    pp_long = pp_state_df['PPLongitude'].loc[pp]
    pp_lat = pp_state_df['PPLatitude'].loc[pp]

    pp_loc = (pp_lat,pp_long)    
    pp_demand.append(pp_state_df['Demand_m3/s'].loc[pp])

    plantcode.append(pp)
    j=0
    for site in bw_state_df.index:
        bw_long = bw_state_df['Longitude'].loc[site]
        bw_lat = bw_state_df['Latitude'].loc[site]

        bw_loc = (bw_lat,bw_long)
        dist_km = hs.haversine(pp_loc,bw_loc)
        
        if dist_km >= 100:
            dist[j,i] = 1e10
        else:
            dist[j,i] = dist_km
        j=j+1

    i=i+1


for site in bw_state_df.index:
    bw_available = bw_state_df['well_yield'].loc[site]*bw_state_df['recovery'].loc[site]/100
    bw_vol.append(bw_available)

    lcow = bw_state_df['lcow'].loc[site]
    LCOW.append(lcow)


def transport_cost(bw_vol,dist):
    transport_lcow_pipe = pipe_costing(bw_available*3600*24,dist_km*1.6)
    transport_lcow_truck = truck_costing(dist_km*1.6)

    transport_lcow = min(transport_lcow_pipe,transport_lcow_truck)
    return float(transport_lcow)


def try_cool_matrix(bw_vol,demand,lcow,dist):

    m = ConcreteModel()
    m.I = pyo.Set(initialize = range(dist.shape[0]))
    # powerplant site index
    m.J = pyo.Set(initialize = range(dist.shape[1]))

    m.matrix= pyo.Set(initialize = m.I*m.J)
    m.x = Var(m.matrix, domain=NonNegativeReals, bounds= (0,1))

    m.constraints= pyo.ConstraintList()

    # Sum of fractions should be less than or equal to 1
    for i in m.I:
        m.constraints.add((sum(m.x[i,j] for j in m.J)) <= 1 )

    # Production to be as close to maximum as possible
    for j in m.J:
       m.constraints.add((sum(m.x[i,j]*float(bw_vol[i]) for i in m.I)) >= demand[j])

    # for j in m.J:
    #     m.constraints.add((sum(( LCOW[i]*m.x[i,j] + transport_cost(bw_vol[i],dist[i,j])*m.x[i,j] ) for i in m.I)) <= 1.4)

    # def obj_demand(m):
    #     return sum(sum(m.x[i,j]*float(bw_vol[i]) for i in m.I) for j in m.J)
    

    # LCOW objective
    def obj(m):
        return sum(sum((LCOW[i]*m.x[i,j] + transport_cost(bw_vol[i],dist[i,j])*m.x[i,j]) for i in m.I) for j in m.J)


    # m.obj2 = Objective(rule=obj_demand,sense = pyo.maximize)
    m.obj1 = Objective(rule = obj)

    solver = SolverFactory('ipopt')
    results = solver.solve(m)
    

    # for constr in m.component_data_objects(ctype=Constraint, active=True, descend_into=True):
    #     tol=1E-6
    #     constr_body_value = value(constr.body, exception=False)
    #     constr_lb_value = value(constr.lower, exception=False)
    #     if constr_body_value is None:
    #             # Undefined constraint body value due to missing variable value
    #         constr_undefined = True
    #         pass
    #     else:
    #         # Check for infeasibilities
    #         if constr.equality:
    #             if fabs(constr_lb_value - constr_body_value) >= tol:
    #                 equality_violated = True
    #                 constr.deactivate()

    # results = solver.solve(m)
    # for constraint in m.

    return m


In [ ]:
solver = SolverFactory('ipopt')

In [ ]:
print(pp_demand)

In [ ]:
def update_demand(supply_max,demand):
    for i in range(0,len(demand)):
        if demand[i]>supply_max:
            demand[i]=supply_max
    return demand

supply_max = sum(bw_vol)/len(bw_vol)

demand = update_demand(supply_max,pp_demand)

m = try_cool_matrix(bw_vol,pp_demand,LCOW,dist)
# m.display()
dict = (m.x.get_values())

x = np.zeros(dist.shape)
print(x)
for i,j in dict.keys():
    x[i,j] = dict[(i,j)]

# print(x)

supply = bw_vol*x[:,0]

# print(sum(supply))
# print(pp_demand[0])
# print(sum(supply)/pp_demand[0]*100)
# print(plantcode[0])

In [ ]:
# figure out how to deactivate infeasible constraints
from idaes.core.util.model_statistics import *

for constr in m.component_data_objects(ctype=Constraint, active=True, descend_into=True):
    tol=1E-6
    constr_body_value = value(constr.body, exception=False)
    constr_lb_value = value(constr.lower, exception=False)
    if constr_body_value is None:
            # Undefined constraint body value due to missing variable value
        constr_undefined = True
        pass
    else:
        # Check for infeasibilities
        if constr.equality:
            if fabs(constr_lb_value - constr_body_value) >= tol:
                equality_violated = True
    constr.deactivate()

In [ ]:
dfx = pd.DataFrame(x)
bw_vol=np.array(bw_vol)
# print((pp_demand))
print(x[:,0])
# log_infeasible_constraints(m)
# print(x.shape)
# print(bw_vol.shape)
# np.matmul(bw_vol,x)

Powerplant % Demand met
1. Assuming multiple brackish water sites provide to a single powerplant
2. Adding transportation cost to LCOW based on the value in variable transport_cost
3. No limitation on maximum distance

Creating a dataframe with one powerplant to many brackish water sites and respective distances.\
Sorting in order of increasing distance.\
Calculating demand met with every brackish water site and weighted tLCOW (LCOW which includes transport costs )

In [ ]:
bw_state_df = bw_df[bw_df['state_alpha']=='TX']
bw_state_df = bw_state_df.set_index('unique_site_ID')

pp_state_df =  pp_combined_df[pp_combined_df['PlantState']=='TX']
pp_state_df = pp_state_df.set_index('PlantCode')

In [ ]:
cols = ['PlantCode','PPLatitude','PPLongitude','PPDemand','BWSites','BWLatitude','BWLongitude','BWAvailable','Distance_km',
        'PPDemandFracMet','LCOW','TransportLCOW','tLCOW','WeightedtLCOW','Cum_PPDemandFracMet']

one_pp_many_bw_df  = pd.DataFrame(columns=cols)

for pp in pp_state_df.index:
    pp_long = pp_state_df['PPLongitude'].loc[pp]
    pp_lat = pp_state_df['PPLatitude'].loc[pp]

    pp_loc = (pp_lat,pp_long)    

    plantcode = pp
    
    for site in bw_state_df.index:
        bw_long = bw_state_df['Longitude'].loc[site]
        bw_lat = bw_state_df['Latitude'].loc[site]

        bw_loc = (bw_lat,bw_long)
        dist_km = hs.haversine(pp_loc,bw_loc)
        
        bw_available = bw_state_df['well_yield'].loc[site]*bw_state_df['recovery'].loc[site]/100
        pp_demand_frac_met = bw_available/pp_state_df['Demand_m3/s'].loc[pp]
        lcow = bw_state_df['lcow'].loc[site] + bw_state_df['pipe_cost'].loc[site] 

        # Updated transport costs
        
        transport_lcow_pipe = pipe_costing(bw_available*3600*24,dist_km)
        transport_lcow_truck = truck_costing(dist_km)

        transport_lcow = min(transport_lcow_pipe,transport_lcow_truck)
        # transport_onm = transport_cost*dist_km*bw_available*365*24*3600
        # tlcow = (lcow*bw_available*365*24*3600 + transport_onm)/(bw_available*365*24*3600) 
        tlcow = lcow + transport_lcow
        
        temp_dict =  {'PlantCode' : pp,
                      'PPLatitude' : pp_lat,
                      'PPLongitude' : pp_long,
                      'PPDemand' : pp_state_df['Demand_m3/s'].loc[pp],
                      'BWSites' : site,
                      'BWLatitude' : bw_lat,
                      'BWLongitude' : bw_long,
                      'BWAvailable' : bw_available,
                      'Distance_km' : dist_km,
                      'PPDemandFracMet' : pp_demand_frac_met,
                      'LCOW' : lcow,
                      'TransportLCOW' : transport_lcow,
                      'tLCOW' : tlcow,
                      'WeightedtLCOW': 0,
                      'Cum_PPDemandFracMet': 0}
        
        temp = pd.DataFrame(temp_dict,index= [0])    
        one_pp_many_bw_df = pd.concat([one_pp_many_bw_df,temp],ignore_index=True)
        one_pp_many_bw_df.reset_index()

one_pp_many_bw_df_sorted1 = one_pp_many_bw_df.sort_values(['PlantCode','tLCOW']).groupby('PlantCode')
one_pp_many_bw_df_sorted2 = one_pp_many_bw_df.sort_values(['PlantCode','Distance_km']).groupby('PlantCode')

one_pp_many_bw_df

In [ ]:
# Using data sorted by tLCOW
one_pp_many_bw_df = one_pp_many_bw_df_sorted1.apply(pd.DataFrame).set_index('PlantCode')
weighted_lcow = []
cum_pp_demand_frac_met = []

for pp in pp_state_df.index:
    cum_bw_available = np.array(one_pp_many_bw_df['BWAvailable'].loc[pp].cumsum())
    cum_cost = np.array((one_pp_many_bw_df['BWAvailable'].loc[pp]*one_pp_many_bw_df['tLCOW'].loc[pp]).cumsum())
    temp = np.divide(cum_cost,cum_bw_available)
    weighted_lcow = np.append(weighted_lcow, temp,axis=0)
    temp_demand_met = np.array(one_pp_many_bw_df['PPDemandFracMet'].loc[pp].cumsum())
    cum_pp_demand_frac_met = np.append(cum_pp_demand_frac_met,temp_demand_met,axis=0)

one_pp_many_bw_df['WeightedtLCOW'] = weighted_lcow
one_pp_many_bw_df['Cum_PPDemandFracMet'] = cum_pp_demand_frac_met
one_pp_many_bw_df['Cum_PPDemandFracMet'] = np.where(one_pp_many_bw_df['Cum_PPDemandFracMet']>1,1,one_pp_many_bw_df['Cum_PPDemandFracMet'])
one_pp_many_bw_df

In [ ]:
# Using data sorted by distance
one_pp_many_bw_df = one_pp_many_bw_df_sorted2.apply(pd.DataFrame).set_index('PlantCode')
weighted_lcow = []
cum_pp_demand_frac_met = []

for pp in pp_state_df.index:
    cum_bw_available = np.array(one_pp_many_bw_df['BWAvailable'].loc[pp].cumsum())
    cum_cost = np.array((one_pp_many_bw_df['BWAvailable'].loc[pp]*one_pp_many_bw_df['tLCOW'].loc[pp]).cumsum())
    temp = np.divide(cum_cost,cum_bw_available)
    weighted_lcow = np.append(weighted_lcow, temp,axis=0)
    temp_demand_met = np.array(one_pp_many_bw_df['PPDemandFracMet'].loc[pp].cumsum())
    cum_pp_demand_frac_met = np.append(cum_pp_demand_frac_met,temp_demand_met,axis=0)

one_pp_many_bw_df['WeightedtLCOW'] = weighted_lcow
one_pp_many_bw_df['Cum_PPDemandFracMet'] = cum_pp_demand_frac_met
one_pp_many_bw_df['Cum_PPDemandFracMet'] = np.where(one_pp_many_bw_df['Cum_PPDemandFracMet']>1,1,one_pp_many_bw_df['Cum_PPDemandFracMet'])
one_pp_many_bw_df.head()

In [ ]:
one_pp_many_bw_df[(one_pp_many_bw_df['PPLongitude']<-105) & (one_pp_many_bw_df['Cum_PPDemandFracMet']>=1)].groupby('PlantCode').first()
# one_pp_many_bw_df[(one_pp_many_bw_df['PPLongitude']<-105)]

Map the maximum % demand that be potentially met with all brackish water sites

In [ ]:
plantcodes = list(one_pp_many_bw_df.groupby('PlantCode').groups.keys())
pp_geo_df.index = plantcodes

In [ ]:
# pp_demand_met_df = one_pp_many_bw_df.groupby('PlantCode').last()
pp_demand_met_df = one_pp_many_bw_df[one_pp_many_bw_df['Cum_PPDemandFracMet']>=1].groupby('PlantCode').first()

fig, (ax0,ax1) = plt.subplots(1,2, figsize = (10,5))
# states[states['NAME'] == 'Texas'].plot(ax=ax,figsize=(20, 20))
texas.plot(ax=ax0,figsize=(50, 50))
pp_geo_df.loc[pp_demand_met_df.index].plot(column = pp_demand_met_df['Cum_PPDemandFracMet'].values*100, ax=ax0, 
                                           markersize = 5,label='PP',legend=True,vmin=0, vmax=100)
ax0.set_title('% Demand met')

texas.plot(ax=ax1,figsize=(50, 50))
pp_geo_df.loc[pp_demand_met_df.index].plot(column = pp_demand_met_df['WeightedtLCOW'].values, 
                                           ax=ax1, markersize = 5, label='PP',legend=True,vmin=0,vmax=3)
ax1.set_title('Average LCOW')

Map the maximum % demand that be potentially met with all brackish water sites
1. Assuming the tLCOW does not exceed maximum LCOW
2. Or 100% demand is met

TODO: In the cases where the adding an additional brackish water site excced maximum LCOW calculate additional fractional supply possible

In [ ]:
plantcodes = list(one_pp_many_bw_df.groupby('PlantCode').groups.keys())
pp_geo_df.index = plantcodes


In [ ]:
one_pp_many_bw_df[(one_pp_many_bw_df['Cum_PPDemandFracMet']>=1)].groupby('PlantCode').first()
one_pp_many_bw_df[(one_pp_many_bw_df['Cum_PPDemandFracMet']<=1)].groupby('PlantCode').last()

In [ ]:
max_LCOW = 1.6
pp_demand_met_conditional_df = one_pp_many_bw_df[(one_pp_many_bw_df['Cum_PPDemandFracMet']>=1) & (one_pp_many_bw_df['WeightedtLCOW']>=max_LCOW)].groupby('PlantCode').first()

fig, (ax0,ax1) = plt.subplots(1,2, figsize = (10,5))
texas.plot(ax=ax0,figsize=(50, 50))
pp_geo_df.loc[pp_demand_met_conditional_df.index].plot(column = pp_demand_met_conditional_df['Cum_PPDemandFracMet'].values*100, ax=ax0,
                                                        markersize = 5, label='PP',legend=True,vmin=0,vmax = 100)
ax0.set_title('% Demand met')

texas.plot(ax=ax1,figsize=(50, 50))
pp_geo_df.loc[pp_demand_met_conditional_df.index].plot(column = pp_demand_met_conditional_df['WeightedtLCOW'].values,
                                                        ax=ax1, markersize = 5, label='PP',legend=True,vmin=0,vmax = 1.6)
ax1.set_title('Average LCOW')

print(len(pp_demand_met_conditional_df.index))

Function to find the most cost effect brackish water sites